In [ ]:
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import traitlets
from jetbot import Robot, Camera, bgr8_to_jpeg
from SCSCtrl import TTLServo
import threading
import time
import os
from datetime import datetime

# ==========================================
# 1. 사용자 라이브러리 임포트 (OCR용)
# ==========================================
# vision 폴더가 notebook과 같은 경로에 있어야 합니다.
import sys
sys.path.append(os.getcwd()) # 현재 경로를 path에 추가
from vision.detector import PlateNumberDetector
from vision.preprocessor import ImagePreprocessor

In [ ]:
# ==========================================
# 2. 전역 변수 및 객체 초기화
# ==========================================

# 2-1. 로봇, 카메라, 서보 초기화
robot = Robot()
camera = Camera.instance(width=224, height=224)
try:
    servo = TTLServo()
except Exception as e:
    print(f"서보 초기화 실패 (시뮬레이션 모드): {e}")
    servo = None

# 2-2. 전역 이미지 버퍼 및 상태 플래그
global_image = np.zeros((224, 224, 3), dtype=np.uint8)
is_running = True          # 전체 시스템 동작 플래그
road_following_active = False # 주행 활성화 플래그 (Start 버튼으로 제어)

# 2-3. 스냅샷 저장 경로 설정
SNAPSHOT_DIR = 'snapshot'
if not os.path.exists(SNAPSHOT_DIR):
    os.makedirs(SNAPSHOT_DIR)

In [ ]:
# ==========================================
# 3. 모델 로드
# ==========================================

# 3-1. Road Following 모델 (ResNet18)
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
# 경로를 실제 모델 파일명으로 수정해주세요
try:
    model.load_state_dict(torch.load('best_steering_model_xy.pth')) 
    device = torch.device('cuda')
    model = model.to(device)
    model = model.eval().half()
    print("Road Following Model Loaded!")
except Exception as e:
    print(f"모델 로드 실패 (경로 확인 필요): {e}")

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

# 3-2. OCR 감지기 초기화 (Clova API)
# .env 파일에 API URL과 KEY가 설정되어 있어야 합니다.
ocr_detector = PlateNumberDetector(
    model="clova",              # ★ 요청사항: paddle 대신 clova 사용
    plate_similarity_thresh=80,
    debug_mode=False
)